In [1]:
import polars as pl
import requests
from bs4 import BeautifulSoup

pl.Config.set_tbl_cols(100000)

polars.config.Config

In [2]:
url = "https://www.fangraphs.com/leaders/major-league?pos={pos}&stats=bat&lg={league}&qual={qual}&type={stat_type}&season={end_season}&season1={start_season}&ind=0&startdate={start_date}&enddate={end_date}&month=0&team=0&pagenum=1&pageitems=2000000000"

# pos options: c, 1b,2b,3b, ss, lf, cf, rf, dh, of, p, all
# qual options: y, n
# league options: "", "al", "nl"
# start date, end date are strings in the format "yyyy-mm-dd"
# stat options: 8 (dashboard), 0 (standard), 1 (advanced), 2 (batted ball), 3 (win_probability), 6 (value), 23 (+stats),24 (statcast), 48 (violations)

In [3]:
def get_table_data(
    stat_type, pos, league, start_date, end_date, qual, start_season, end_season
):
    # Assuming `cont` contains the HTML content
    cont = requests.get(
        url.format(
            pos=pos,
            league=league,
            stat_type=stat_type,
            start_season=start_season,
            end_season=end_season,
            qual=qual,
            start_date=start_date,
            end_date=end_date,
        )
    ).content.decode("utf-8")

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(cont, "html.parser")

    # Find the main table using the provided CSS selector
    main_table = soup.select_one(
        "#content > div.leaders-major_leaders-major__table__hcmbm > div.fg-data-grid.table-type > div.table-wrapper-outer > div > div.table-scroll > table"
    )

    # Find the table header
    thead = main_table.find("thead")

    # Extract column names from the data-col-id attribute of the <th> elements, excluding "divider"
    headers = [
        th["data-col-id"]
        for th in thead.find_all("th")
        if "data-col-id" in th.attrs and th["data-col-id"] != "divider"
    ]

    # Find the table body within the main table
    tbody = main_table.find("tbody")

    # Initialize a list to store the extracted data
    data = []

    # Iterate over each row in the table body
    for row in tbody.find_all("tr"):
        row_data = {header: None for header in headers}  # Initialize with None
        for cell in row.find_all("td"):
            col_id = cell.get("data-col-id")

            if col_id and col_id != "divider":
                if cell.find("a"):
                    row_data[col_id] = cell.find("a").text
                elif cell.find("span"):
                    row_data[col_id] = cell.find("span").text
                else:
                    text = cell.text.strip().replace("%", "")
                    if text == "":
                        row_data[col_id] = None
                    else:
                        try:
                            row_data[col_id] = float(text) if "." in text else int(text)
                        except ValueError:
                            row_data[col_id] = text
                        except Exception as e:
                            print(e)
                            print(cell.attrs["data-col-id"])
                            row_data[col_id] = text
        # Print row_data for debugging
        data.append(row_data)

    # Create a Polars DataFrame from the extracted data
    df = pl.DataFrame(data, infer_schema_length=None)
    return df

In [4]:
# Define the available stat types as an Enum
from enum import Enum


class FangraphsBattingStatType(Enum):
    DASHBOARD = 8
    STANDARD = 0
    ADVANCED = 1
    BATTED_BALL = 2
    WIN_PROBABILITY = 3
    VALUE = 6
    PLUS_STATS = 23
    STATCAST = 24
    VIOLATIONS = 48
    SPORTS_INFO_PITCH_TYPE = 4
    SPORTS_INFO_PITCH_VALUE = 7
    SPORTS_INFO_PLATE_DISCIPLINE = 5
    STATCAST_PITCH_TYPE = 9
    STATCAST_VELO = 10
    STATCAST_H_MOVEMENT = 11
    STATCAST_V_MOVEMENT = 12
    STATCAST_PITCH_TYPE_VALUE = 13
    STATCAST_PITCH_TYPE_VALUE_PER_100 = 14
    STATCAST_PLATE_DISCIPLINE = 15

In [5]:
stat_types = {}
for stat_type in FangraphsBattingStatType:
    stat_types[stat_type] = stat_type.value

In [6]:
df_list = []
for stat_type in stat_types:
    print(f"Fetching data for {stat_type}...")
    df = get_table_data(
        stat_types[stat_type], "all", "", "2024-04-01", "2024-05-01", "y", "", ""
    )
    if df is not None:
        print(f"Data fetched for {stat_type}")
        df_list.append(df)
    else:
        print(f"Warning: No data returned for {stat_type}")

Fetching data for FangraphsBattingStatType.DASHBOARD...
Data fetched for FangraphsBattingStatType.DASHBOARD
Fetching data for FangraphsBattingStatType.STANDARD...
Data fetched for FangraphsBattingStatType.STANDARD
Fetching data for FangraphsBattingStatType.ADVANCED...
Data fetched for FangraphsBattingStatType.ADVANCED
Fetching data for FangraphsBattingStatType.BATTED_BALL...
Data fetched for FangraphsBattingStatType.BATTED_BALL
Fetching data for FangraphsBattingStatType.WIN_PROBABILITY...
Data fetched for FangraphsBattingStatType.WIN_PROBABILITY
Fetching data for FangraphsBattingStatType.VALUE...
Data fetched for FangraphsBattingStatType.VALUE
Fetching data for FangraphsBattingStatType.PLUS_STATS...
Data fetched for FangraphsBattingStatType.PLUS_STATS
Fetching data for FangraphsBattingStatType.STATCAST...
Data fetched for FangraphsBattingStatType.STATCAST
Fetching data for FangraphsBattingStatType.VIOLATIONS...
Data fetched for FangraphsBattingStatType.VIOLATIONS
Fetching data for Fang

In [7]:
from polars import selectors as cs

df = df_list[0]
len_cols = len(df.columns)
for i in range(1, len(df_list)):
    len_cols += len(df_list[i].columns)
    df = df.join(df_list[i], on="Name", how="inner").select(
        ~cs.ends_with("_right"),
    )
df.head()

Name,Team,G,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP,AVG,OBP,SLG,wOBA,xwOBA,wRC+,BaseRunning,Offense,Defense,WAR,AB,H,1B,2B,3B,BB,IBB,SO,HBP,SF,SH,GDP,CS,BB/K,OPS,Spd,UBR,GDPRuns,XBR,wBsR,wRC,wRAA,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH,IFH%,BUH,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,Batting,Fielding,Positional,wLeague,Replacement,RAR,Dollars,BB%+,K%+,AVG+,OBP+,SLG+,ISO+,BABIP+,LD%+,GB%+,FB%+,Pull%+,Cent%+,Oppo%+,Events,EV,maxEV,LA,Barrels,Barrel%,HardHit,HardHit%,xAVG,xSLG,PPTV,CPTV,DGV,DSV,BPTV,BTV,rPPTV,rCPTV,rDGV,rDSV,rBPTV,rBTV,EBV,ESV,rFTeamV,rBTeamV,rTV,FB%1,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,CStr%,C+SwStr%,pfxFA%,pfxFT%,pfxFC%,pfxFS%,pfxFO%,pfxSI%,pfxSL%,pfxCU%,pfxKC%,pfxEP%,pfxCH%,pfxSC%,pfxKN%,pfxUN%,pfxvFA,pfxvFT,pfxvFC,pfxvFS,pfxvFO,pfxvSI,pfxvSL,pfxvCU,pfxvKC,pfxvEP,pfxvCH,pfxvSC,pfxvKN,pfxFA-X,pfxFT-X,pfxFC-X,pfxFS-X,pfxFO-X,pfxSI-X,pfxSL-X,pfxCU-X,pfxKC-X,pfxEP-X,pfxCH-X,pfxSC-X,pfxKN-X,pfxFA-Z,pfxFT-Z,pfxFC-Z,pfxFS-Z,pfxFO-Z,pfxSI-Z,pfxSL-Z,pfxCU-Z,pfxKC-Z,pfxEP-Z,pfxCH-Z,pfxSC-Z,pfxKN-Z,pfxwFA,pfxwFT,pfxwFC,pfxwFS,pfxwFO,pfxwSI,pfxwSL,pfxwCU,pfxwKC,pfxwEP,pfxwCH,pfxwSC,pfxwKN,pfxwFA/C,pfxwFT/C,pfxwFC/C,pfxwFS/C,pfxwFO/C,pfxwSI/C,pfxwSL/C,pfxwCU/C,pfxwKC/C,pfxwEP/C,pfxwCH/C,pfxwSC/C,pfxwKN/C,pfxO-Swing%,pfxZ-Swing%,pfxSwing%,pfxO-Contact%,pfxZ-Contact%,pfxContact%,pfxZone%,pfxPace
str,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,f64,null,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Josh Naylor""","""CLE""",152,633,31,84,108,6,9.2,16.6,0.213,0.246,0.243,0.32,0.456,0.332,0.335,118,-2.5,10.7,-9.9,2.3,563,137,79,27,0,58,7,105,7,4,0,17,2,0.55,0.776,3.2,-2.3,-2.1,-1.8,-0.7,85,10.9,1.29,17.1,46.8,36.1,10.8,18.6,5,2.3,0,0.0,41.6,37.4,21.0,16.0,52.6,31.4,0.52,-11.62,12.14,14.6,1.46,1.0,0.41,1,1.28,-0.76,13.2,1.9,-11.8,2.0,19.1,21.9,"""$18.1""",113,73,102,103,116,138,86,88,113,93,102,107,87,463,89.9,110.9,11.0,39,8.4,189,40.8,0.257,0.443,5,0,0,0,0,0,0.4,null,null,null,null,null,5,0,0.4,0.0,0.4,39.7,93.5,20.1,84.1,8.5,89.3,10.1,80.8,16.4,86.2,4.9,87.5,0.4,80.3,1.2,5.4,1.1,-0.4,1.7,1.1,-1.7,-0.3,0.6,0.25,-0.21,0.75,0.3,-1.53,-3.19,35.5,73.7,50.3,69.8,86.7,79.4,38.8,59.9,10.4,12.6,23.0,27.2,null,8.6,4.6,null,12.2,19.1,8.1,3.0,0.3,16.7,null,0.4,null,94.1,null,89.3,87.2,null,93.1,84.2,80.0,82.6,51.7,86.2,null,80.2,-2.3,null,0.4,-6.0,null,0.3,0.8,2.9,3.1,1.3,-7.7,null,-2.9,9.1,null,4.3,2.2,null,4.8,1.0,-4.5,-5.5,7.6,2.7,null,-2.8,10.2,null,-0.9,-1.1,null,-3.7,1.3,-1.5,2.5,-0.4,0.2,null,-0.3,1.65,null,-0.44,-1.05,null,-1.33,0.3,-0.8,3.75,-7.41,0.06,null,-3.22,32.1,70.8,50.3,64.7,86.7,79.3,47.1,20.1
"""Christian Walker""","""ARI""",130,552,26,72,84,2,10.0,24.1,0.217,0.287,0.251,0.335,0.468,0.343,0.351,119,-3.2,9.4,1.3,3.0,479,120,68,26,0,55,4,133,10,8,0,12,2,0.41,0.803,2.2,1.2,-0.2,-1.8,-1.4,79,14.8,0.81,15.3,37.9,46.9,12.7,15.7,6,4.5,0,0.0,42.1,35.3,22.6,14.4,46.9,38.7,0.14,-9.34,9.48,6.37,0.67,0.96,1.45,3,0.95,-0.81,12.7,10.8,-9.5,1.7,16.7,29.1,"""$24.1""",121

In [9]:
df.shape

(129, 244)